# 제품 이상여부 판별 프로젝트 (본선)


## 1. 데이터 불러오기


### 필수 라이브러리


In [ ]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 40.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 16.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import catboost
from catboost import CatBoostClassifier , Pool
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
import random
import os

# 랜덤 시드를 고정
random.seed(42)
np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

### 데이터 읽어오기


In [ ]:
ROOT_DIR = "data"
RANDOM_STATE = 42

# Load data
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
submission = pd.read_csv(os.path.join("submission.csv"))

## 2. 수기 데이터 읽어오기

수기 데이터는 제조 환경과 관련된 정보들이 담겨있는 데이터입니다.

In [ ]:
df_hand = pd.read_excel(os.path.join(ROOT_DIR, "hand_data.xlsx"))
df_hand

,day,start_time,end_time,Dam Thickness - 1time / day,Fill thickness - 1time / day,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time,Line Temp. - Every break time,Line humidity - Every break time,OCR gap - Line#1 stage1,OCR gap - Line#1 stage2,OCR gap - Line#1 stage3,OCR gap - Line #2 stage1,OCR gap - Line #2 stage2,OCR gap - Line #2 stage3
0,240319,08:00:00,10:30:00,454.0,193.0,1728.0,2919.0,23.4,56.6,1.2,1.000,1.200,1.2,0.900,1.4
1,240319,10:45:00,12:00:00,454.0,193.0,1754.0,2962.0,24.0,50.9,1.2,1.000,1.200,1.2,0.900,1.4
2,240319,13:00:00,15:00:00,454.0,193.0,1708.0,2887.0,24.5,49.6,1.2,1.000,1.200,1.2,0.900,1.4
3,240319,15:15:00,18:00:00,454.0,193.0,1751.0,2946.0,24.7,53.4,1.2,1.000,1.200,1.2,1.200,1.4
4,240319,18:30:00,20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,240428,08:00:00,10:30:00,478.0,232.0,1642.0,2952.0,24.6,53.0,1.0,1.232,1.466,1.2,1.167,1.5
101,240428,10:45:00,12:00:00,478.0,232.0,1679.0,2942.0,24.8,52.7,1.0,1.232,1.466,1.2,1.167,1.5
102,240428,13:00:00,15:00:00,478.0,232.0,1552.0,2941.0,24.6,55.5,1.0,1.232,1.466,1.2,1.167,1.5
103,240428,15:15:00,18:00:00,478.0,232.0,1552.0,2934.0,24.3,54.6,1.0,1.232,1.466,1.2,1.167,1.5


In [ ]:
# 남길 컬럼 리스트
columns_to_keep = ['day', 'start_time', 'end_time', 'Line Temp. - Every break time', 'Line humidity - Every break time', 'Fill cure energy (mJ) - Every break time', 'Fill cure power (mW) - Every break time']

# 해당 컬럼들만 남기고 나머지는 제거
df_hand = df_hand[columns_to_keep]


In [ ]:
df_hand = df_hand.dropna()

df_hand

,day,start_time,end_time,Line Temp. - Every break time,Line humidity - Every break time,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time
0,240319,08:00:00,10:30:00,23.4,56.6,1728.0,2919.0
1,240319,10:45:00,12:00:00,24.0,50.9,1754.0,2962.0
2,240319,13:00:00,15:00:00,24.5,49.6,1708.0,2887.0
3,240319,15:15:00,18:00:00,24.7,53.4,1751.0,2946.0
5,240320,08:00:00,10:30:00,24.2,55.6,1739.0,2875.0
...,...,...,...,...,...,...,...
100,240428,08:00:00,10:30:00,24.6,53.0,1642.0,2952.0
101,240428,10:45:00,12:00:00,24.8,52.7,1679.0,2942.0
102,240428,13:00:00,15:00:00,24.6,55.5,1552.0,2941.0
103,240428,15:15:00,18:00:00,24.3,54.6,1552.0,2934.0


In [ ]:
df_hand['Energy_per_Power'] = df_hand['Fill cure energy (mJ) - Every break time'] / df_hand['Fill cure power (mW) - Every break time']

In [ ]:
df_hand = df_hand.drop('Fill cure energy (mJ) - Every break time', axis = 1)
df_hand = df_hand.drop('Fill cure power (mW) - Every break time', axis = 1)


In [ ]:
drop_cols = []
for i in range(len(train.columns)):
    if len(train.iloc[:, i].unique()) == 1:
        drop_cols.append(train.columns[i])
train = train.drop(drop_cols, axis=1)

In [ ]:
drop_cols = []
for column in train.columns:
  if (train[column].notnull().sum() // 5) < train[
      column].isnull().sum():
      drop_cols.append(column)
train[drop_cols] = train[drop_cols].fillna(0)
train = train
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,240.0,2.5,-90,100,1030,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,240.0,2.5,-90,70,1030,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1000.0,12.5,90,85,280,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1000.0,12.5,90,70,280,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,240.0,2.5,-90,70,1030,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,2023-09-14 15:50:00,240.0,2.5,-90,70,1030,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,2024-04-10 14:20:00,1000.0,12.5,90,100,280,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,2024-02-22 08:40:00,240.0,2.5,-90,100,1030,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,2023-07-25 11:00:00,1000.0,12.5,90,70,280,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [ ]:
real_train = train.T.drop_duplicates().T
common_columns = real_train.columns.intersection(train.columns)
real_train = real_train.astype(train[common_columns].dtypes)
real_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 129 entries, Equipment_Dam to target
dtypes: float64(66), int64(48), object(15)
memory usage: 39.9+ MB


In [ ]:
train = real_train

In [ ]:
test = test[train.columns]

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE END POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE END POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'HEAD Standby Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['HEAD Standby Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'Head Clean Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['Head Clean Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE START POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE START POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE SPEED Collect', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE SPEED Collect'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,...,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,425.5,148,2050,16,14.9,8.4,...,428.0,243.7,91.8,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,425.5,120,2050,10,21.3,4.9,...,243.7,243.7,270.0,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1375.5,135,1300,16,14.7,8.5,...,1324.2,243.5,91.8,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1375.5,120,1300,10,21.3,8.4,...,243.5,243.5,270.0,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,424.5,120,2050,10,9.7,4.9,...,243.7,243.7,270.0,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,2023-09-14 15:50:00,425.5,120,2050,10,17.0,5.0,...,243.7,243.7,270.0,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,2024-04-10 14:20:00,1375.5,148,1300,16,14.9,8.5,...,1324.2,243.5,91.8,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,2024-02-22 08:40:00,425.5,150,2050,16,14.2,8.2,...,428.0,243.7,91.8,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,2023-07-25 11:00:00,1375.5,120,1300,10,9.7,4.9,...,243.5,243.5,270.0,85,20.100,13.0,117,1,0,Normal


### TEST

In [ ]:
drop_cols = []
for i in range(len(test.columns)):
    if len(test.iloc[:,i].unique()) == 1:
        drop_cols.append(test.columns[i])
test = test.drop(drop_cols, axis=1)

In [ ]:
    # Drop columns with more than half of the values missing
    drop_cols = []
    for column in test.columns:
        if (test[column].notnull().sum() // 5) < test[
            column
        ].isnull().sum():
            drop_cols.append(column)

    test[drop_cols] = test[drop_cols].fillna(0)

    test = test

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE END POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE END POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'HEAD Standby Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['HEAD Standby Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'Head Clean Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['Head Clean Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE START POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE START POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE SPEED Collect', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE SPEED Collect'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,...,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,Dam dispenser #2,AJX75334501,3J1XF767-1,2023-09-15 13:20:00,1375.5,120,1300,10,17.0,4.9,...,1324.2,243.5,243.5,270.0,85,19.8,13.0,195,1,0
1,Dam dispenser #2,AJX75334501,4B1XD472-2,2024-02-06 16:50:00,1375.5,120,1300,16,14.2,8.3,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,14,256,1
2,Dam dispenser #1,AJX75334501,3H1XE355-1,2023-07-14 11:30:00,425.5,120,2050,10,9.7,4.9,...,428.0,243.7,243.7,270.0,85,19.7,1.0,98,1,0
3,Dam dispenser #2,AJX75334501,3L1XA128-1,2023-11-03 08:00:00,1375.5,120,1300,10,21.3,10.6,...,1324.2,1324.2,243.5,91.8,50,85.0,20.0,14,0,1
4,Dam dispenser #1,AJX75334501,4A1XA639-1,2023-12-23 14:00:00,425.5,120,2050,16,13.2,7.5,...,427.9,428.0,243.7,91.8,50,85.0,19.8,1,215,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,2023-10-18 11:10:00,1375.5,120,1300,10,21.3,10.6,...,1324.2,1324.2,243.5,91.8,50,85.0,19.5,14,131,1
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,2024-01-02 15:00:00,1375.5,120,1300,16,13.2,7.6,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,12,279,1
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,2023-11-15 09:10:00,425.5,120,2050,16,13.2,6.6,...,427.9,428.0,243.7,91.8,50,85.0,20.5,4,66,1
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,2023-05-18 14:20:00,424.5,121,2050,10,9.7,3.9,...,428.0,243.7,243.7,270.0,85,18.9,1.0,117,1,0


### 피쳐엔지니어링

In [ ]:
# 불쾌지수 계산 공식 함수
def calculate_discomfort_index(temperature, humidity):
    return 0.81 * temperature + 0.01 * humidity * (0.99 * temperature - 14.3) + 46.3

In [ ]:
train['Collect Date_Fill2'] = pd.to_datetime(train['Collect Date_Fill2'])
df_hand['day'] = pd.to_datetime(df_hand['day'], format='%y%m%d')

df_hand['start_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['start_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df_hand['end_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['end_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

In [ ]:
# train_data의 각 row에 대해 df_hand의 시간 범위와 비교하여 매칭된 행 병합
def match_time(row, df_hand):
    collect_time = row['Collect Date_Fill2']

    # 해당 collect_time이 df_hand의 start_time과 end_time 사이에 있는 행
    mask = (df_hand['start_time'] <= collect_time) & (df_hand['end_time'] >= collect_time)

    # 매칭된 행이 있으면 해당 행을 반환, 없으면 결측치 반환
    if mask.any():
        return df_hand[mask].iloc[0]  # 첫 번째 매칭된 행 반환
    else:
        return pd.Series([None] * len(df_hand.columns), index=df_hand.columns)  # 결측치 반환

# train_data의 각 행에 대해 df_hand에서 매칭되는 행 추가
df_hand_matched = train.apply(lambda row: match_time(row, df_hand), axis=1)

# train_data와 df_hand_matched를 병합
train = pd.concat([train.reset_index(drop=True), df_hand_matched.reset_index(drop=True)], axis=1)


In [ ]:
train['discomfort_index'] = train.apply(lambda row: calculate_discomfort_index(
    row['Line Temp. - Every break time'],
    row['Line humidity - Every break time']), axis=1)

In [ ]:
train = train.drop('end_time', axis = 1)
train = train.drop('start_time', axis = 1)
train = train.drop('day', axis = 1)
# train = train.drop('Line Temp. - Every break time', axis = 1)
# train = train.drop('Line humidity - Every break time', axis = 1)


In [ ]:
test['Collect Date_Fill2'] = pd.to_datetime(test['Collect Date_Fill2'])
# df_hand['day'] = pd.to_datetime(df_hand['day'], format='%y%m%d')

# df_hand['start_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['start_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
# df_hand['end_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['end_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

In [ ]:
# train_data의 각 row에 대해 df_hand의 시간 범위와 비교하여 매칭된 행 병합
def match_time(row, df_hand):
    collect_time = row['Collect Date_Fill2']

    # 해당 collect_time이 df_hand의 start_time과 end_time 사이에 있는 행
    mask = (df_hand['start_time'] <= collect_time) & (df_hand['end_time'] >= collect_time)

    # 매칭된 행이 있으면 해당 행을 반환, 없으면 결측치 반환
    if mask.any():
        return df_hand[mask].iloc[0]  # 첫 번째 매칭된 행 반환
    else:
        return pd.Series([None] * len(df_hand.columns), index=df_hand.columns)  # 결측치 반환

# train_data의 각 행에 대해 df_hand에서 매칭되는 행 추가
df_hand_matched = test.apply(lambda row: match_time(row, df_hand), axis=1)

# train_data와 df_hand_matched를 병합
test = pd.concat([test.reset_index(drop=True), df_hand_matched.reset_index(drop=True)], axis=1)


In [ ]:
test['discomfort_index'] = test.apply(lambda row: calculate_discomfort_index(
    row['Line Temp. - Every break time'],
    row['Line humidity - Every break time']), axis=1)

In [ ]:
test = test.drop('end_time', axis = 1)
test = test.drop('start_time', axis = 1)
test = test.drop('day', axis = 1)
# test = test.drop('Line Temp. - Every break time', axis = 1)
# test = test.drop('Line humidity - Every break time', axis = 1)

In [ ]:
# 시간 정보를 처리할 열 목록
time_columns = ['Collect Date_Dam', 'Collect Date_Fill1', 'Collect Date_Fill2', 'Collect Date_AutoClave']

# train 데이터에서 각 열에 대해 년, 월, 일, 시간, 분 정보 추출 후 원본 열 삭제
for col in time_columns:
    train[col] = pd.to_datetime(train[col])
    train[f'{col}_Year'] = train[col].dt.year
    train[f'{col}_Month'] = train[col].dt.month
    train[f'{col}_Day'] = train[col].dt.day
    train[f'{col}_Hour'] = train[col].dt.hour
    train[f'{col}_Minute'] = train[col].dt.minute

# 원래 있던 시간 정보 열 삭제
train.drop(columns=time_columns, inplace=True)

# test 데이터에서도 동일한 작업 수행
for col in time_columns:
    test[col] = pd.to_datetime(test[col])
    test[f'{col}_Year'] = test[col].dt.year
    test[f'{col}_Month'] = test[col].dt.month
    test[f'{col}_Day'] = test[col].dt.day
    test[f'{col}_Hour'] = test[col].dt.hour
    test[f'{col}_Minute'] = test[col].dt.minute

# 원래 있던 시간 정보 열 삭제
test.drop(columns=time_columns, inplace=True)

In [ ]:
# '-'를 기준으로 앞부분과 뒷부분을 분리
train['workorder'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[0])
train['workorder_num'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

test['workorder'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[0])
test['workorder_num'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

In [ ]:
train['1st Pressure'] = train['1st Pressure 1st Pressure Unit Time_AutoClave'] / train['1st Pressure Collect Result_AutoClave']
train['2nd Pressure'] = train['2nd Pressure Unit Time_AutoClave'] / train['2nd Pressure Collect Result_AutoClave']
train['3rd Pressure'] = train['3rd Pressure Unit Time_AutoClave'] / train['3rd Pressure Collect Result_AutoClave']
train['Chamber Temp AutoClave'] = train['Chamber Temp. Unit Time_AutoClave'] / train['Chamber Temp. Collect Result_AutoClave']

test['1st Pressure'] = test['1st Pressure 1st Pressure Unit Time_AutoClave'] / test['1st Pressure Collect Result_AutoClave']
test['2nd Pressure'] = test['2nd Pressure Unit Time_AutoClave'] / test['2nd Pressure Collect Result_AutoClave']
test['3rd Pressure'] = test['3rd Pressure Unit Time_AutoClave'] / test['3rd Pressure Collect Result_AutoClave']
test['Chamber Temp AutoClave'] = test['Chamber Temp. Unit Time_AutoClave'] / test['Chamber Temp. Collect Result_AutoClave']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Dispense Volume Collect Result_Fill1'] = train['Dispense Volume(Stage1) Collect Result_Fill1'] + train['Dispense Volume(Stage2) Collect Result_Fill1'] + train['Dispense Volume(Stage3) Collect Result_Fill1']

test['Total Dispense Volume Collect Result_Fill1'] = test['Dispense Volume(Stage1) Collect Result_Fill1'] + test['Dispense Volume(Stage2) Collect Result_Fill1'] + test['Dispense Volume(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Process Time'] = train['Machine Tact time Collect Result_Dam'] + train['Machine Tact time Collect Result_Fill1'] + train['Machine Tact time Collect Result_Fill2']

test['Total Process Time'] = test['Machine Tact time Collect Result_Dam'] + test['Machine Tact time Collect Result_Fill1'] + test['Machine Tact time Collect Result_Fill2']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Circle Result'] = train['Stage1 Circle2 Distance Speed Collect Result_Dam'] + train['Stage1 Circle1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Circle Result'] = test['Stage1 Circle2 Distance Speed Collect Result_Dam'] + test['Stage1 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage2 Circle Result'] = train['Stage2 Circle2 Distance Speed Collect Result_Dam'] + train['Stage2 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage2 Circle Result'] = test['Stage2 Circle2 Distance Speed Collect Result_Dam'] + test['Stage2 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage3 Circle Result'] = train['Stage3 Circle2 Distance Speed Collect Result_Dam'] + train['Stage3 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage3 Circle Result'] = test['Stage3 Circle2 Distance Speed Collect Result_Dam'] + test['Stage3 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
train['Total Production'] = train['Production Qty Collect Result_Dam'] + train['Production Qty Collect Result_Fill1'] + train['Production Qty Collect Result_Fill2']

test['Total Production'] = test['Production Qty Collect Result_Dam'] + test['Production Qty Collect Result_Fill1'] + test['Production Qty Collect Result_Fill2']

In [ ]:
train['Workorder_Dam_Middle'] = train['Workorder_Dam'].str.slice(3, 5)

test['Workorder_Dam_Middle'] = test['Workorder_Dam'].str.slice(3, 5)

In [ ]:
train['Workorder_Dam_First'] = train['Workorder_Dam'].str.slice(0, 2)

test['Workorder_Dam_First'] = test['Workorder_Dam'].str.slice(0, 2)

In [ ]:
# 변경할 열들의 리스트
columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 열의 문자 값을 0으로 변경
train[columns_to_replace] = train[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)
test[columns_to_replace] = test[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)

In [ ]:
train.drop(['workorder'], axis=1, inplace=True)

test.drop(['workorder'], axis=1, inplace=True)

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Line Result'] = train['Stage1 Line2 Distance Speed Collect Result_Dam'] + train['Stage1 Line1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Line Result'] = test['Stage1 Line2 Distance Speed Collect Result_Dam'] + test['Stage1 Line1 Distance Speed Collect Result_Dam']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage2 Line Result'] = train['Stage2 Line2 Distance Speed Collect Result_Dam'] + train['Stage2 Line3 Distance Speed Collect Result_Dam'] + train['Stage2 Line4 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage2 Line Result'] = test['Stage2 Line2 Distance Speed Collect Result_Dam'] + test['Stage2 Line3 Distance Speed Collect Result_Dam'] + train['Stage2 Line4 Distance Speed Collect Result_Dam']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage3 Line Result'] = train['Stage3 Line2 Distance Speed Collect Result_Dam'] + train['Stage3 Line1 Distance Speed Collect Result_Dam'] + train['Stage3 Line4 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage3 Line Result'] = test['Stage3 Line2 Distance Speed Collect Result_Dam'] + test['Stage3 Line1 Distance Speed Collect Result_Dam'] + train['Stage3 Line4 Distance Speed Collect Result_Dam']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

In [ ]:
# 기존 코드 유지
categorical_feat = train.columns.tolist()
len(categorical_feat)

categorical_feat.remove('target')

train[categorical_feat] = train[categorical_feat].astype('str')
test[categorical_feat] = test[categorical_feat].astype('str')

# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols = categorical_feat

In [ ]:
train[categorical_feat] = train[categorical_feat].astype('str')
test[categorical_feat] = test[categorical_feat].astype('str')

In [ ]:
# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols = categorical_feat

In [ ]:
train['target'] = train['target'].replace({'Normal': 0, 'AbNormal': 1})

In [ ]:
stratify_columns = train[["target", "Workorder_Dam_Middle"]]

In [ ]:
# 데이터를 stratified split
x_trn, x_val = train_test_split(
    train, test_size=0.2, random_state=42, stratify=stratify_columns)

In [ ]:
y_trn = x_trn['target']
x_trn = x_trn.drop(['target'], axis=1)

y_val = x_val['target']
x_val = x_val.drop(['target'], axis=1)

In [ ]:
train2 = train.copy()
test2 = test.copy()

### Data_2

In [ ]:
# Load data
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
submission = pd.read_csv(os.path.join("submission.csv"))

In [ ]:
drop_cols = []
for i in range(len(train.columns)):
    if len(train.iloc[:, i].unique()) == 1:
        drop_cols.append(train.columns[i])
train = train.drop(drop_cols, axis=1)

In [ ]:
drop_cols = []
for column in train.columns:
  if (train[column].notnull().sum() // 5) < train[
      column].isnull().sum():
      drop_cols.append(column)
train[drop_cols] = train[drop_cols].fillna(0)
train = train

In [ ]:
real_train = train.T.drop_duplicates().T
common_columns = real_train.columns.intersection(train.columns)
real_train = real_train.astype(train[common_columns].dtypes)
real_train.info()

In [ ]:
train = real_train

In [ ]:
test = test[train.columns]

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE END POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE END POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'HEAD Standby Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['HEAD Standby Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'Head Clean Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['Head Clean Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE START POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE START POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE SPEED Collect', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE SPEED Collect'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

### Test

In [ ]:
drop_cols = []
for i in range(len(test.columns)):
    if len(test.iloc[:,i].unique()) == 1:
        drop_cols.append(test.columns[i])
test = test.drop(drop_cols, axis=1)

In [ ]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in test.columns:
    if (test[column].notnull().sum() // 5) < test[
        column
    ].isnull().sum():
        drop_cols.append(column)

test[drop_cols] = test[drop_cols].fillna(0)

test = test

In [ ]:
# 불쾌지수 계산 공식 함수
def calculate_discomfort_index(temperature, humidity):
    return 0.81 * temperature + 0.01 * humidity * (0.99 * temperature - 14.3) + 46.3

In [ ]:
train['Collect Date_Fill2'] = pd.to_datetime(train['Collect Date_Fill2'])
# df_hand['day'] = pd.to_datetime(df_hand['day'], format='%y%m%d')

# df_hand['start_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['start_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
# df_hand['end_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['end_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

In [ ]:
# train_data의 각 row에 대해 df_hand의 시간 범위와 비교하여 매칭된 행 병합
def match_time(row, df_hand):
    collect_time = row['Collect Date_Fill2']

    # 해당 collect_time이 df_hand의 start_time과 end_time 사이에 있는 행
    mask = (df_hand['start_time'] <= collect_time) & (df_hand['end_time'] >= collect_time)

    # 매칭된 행이 있으면 해당 행을 반환, 없으면 결측치 반환
    if mask.any():
        return df_hand[mask].iloc[0]  # 첫 번째 매칭된 행 반환
    else:
        return pd.Series([None] * len(df_hand.columns), index=df_hand.columns)  # 결측치 반환

# train_data의 각 행에 대해 df_hand에서 매칭되는 행 추가
df_hand_matched = train.apply(lambda row: match_time(row, df_hand), axis=1)

# train_data와 df_hand_matched를 병합
train = pd.concat([train.reset_index(drop=True), df_hand_matched.reset_index(drop=True)], axis=1)


In [ ]:
train['discomfort_index'] = train.apply(lambda row: calculate_discomfort_index(
    row['Line Temp. - Every break time'],
    row['Line humidity - Every break time']), axis=1)

In [ ]:
train = train.drop('end_time', axis = 1)
train = train.drop('start_time', axis = 1)
train = train.drop('day', axis = 1)
# train = train.drop('Line Temp. - Every break time', axis = 1)
# train = train.drop('Line humidity - Every break time', axis = 1)

In [ ]:
test['Collect Date_Fill2'] = pd.to_datetime(test['Collect Date_Fill2'])
# df_hand['day'] = pd.to_datetime(df_hand['day'], format='%y%m%d')

# df_hand['start_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['start_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
# df_hand['end_time'] = pd.to_datetime(df_hand['day'].astype(str) + ' ' + df_hand['end_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

In [ ]:
# train_data의 각 row에 대해 df_hand의 시간 범위와 비교하여 매칭된 행 병합
def match_time(row, df_hand):
    collect_time = row['Collect Date_Fill2']

    # 해당 collect_time이 df_hand의 start_time과 end_time 사이에 있는 행
    mask = (df_hand['start_time'] <= collect_time) & (df_hand['end_time'] >= collect_time)

    # 매칭된 행이 있으면 해당 행을 반환, 없으면 결측치 반환
    if mask.any():
        return df_hand[mask].iloc[0]  # 첫 번째 매칭된 행 반환
    else:
        return pd.Series([None] * len(df_hand.columns), index=df_hand.columns)  # 결측치 반환

# train_data의 각 행에 대해 df_hand에서 매칭되는 행 추가
df_hand_matched = test.apply(lambda row: match_time(row, df_hand), axis=1)

# train_data와 df_hand_matched를 병합
test = pd.concat([test.reset_index(drop=True), df_hand_matched.reset_index(drop=True)], axis=1)


In [ ]:
test['discomfort_index'] = test.apply(lambda row: calculate_discomfort_index(
    row['Line Temp. - Every break time'],
    row['Line humidity - Every break time']), axis=1)

In [ ]:
test = test.drop('end_time', axis = 1)
test = test.drop('start_time', axis = 1)
test = test.drop('day', axis = 1)
# test = test.drop('Line Temp. - Every break time', axis = 1)
# test = test.drop('Line humidity - Every break time', axis = 1)

In [ ]:
# 시간 정보를 처리할 열 목록
time_columns = ['Collect Date_Dam', 'Collect Date_Fill1', 'Collect Date_Fill2', 'Collect Date_AutoClave']

# train 데이터에서 각 열에 대해 년, 월, 일, 시간, 분 정보 추출 후 원본 열 삭제
for col in time_columns:
    train[col] = pd.to_datetime(train[col])
    train[f'{col}_Year'] = train[col].dt.year
    train[f'{col}_Month'] = train[col].dt.month
    train[f'{col}_Day'] = train[col].dt.day
    train[f'{col}_Hour'] = train[col].dt.hour
    train[f'{col}_Minute'] = train[col].dt.minute

# 원래 있던 시간 정보 열 삭제
train.drop(columns=time_columns, inplace=True)

# test 데이터에서도 동일한 작업 수행
for col in time_columns:
    test[col] = pd.to_datetime(test[col])
    test[f'{col}_Year'] = test[col].dt.year
    test[f'{col}_Month'] = test[col].dt.month
    test[f'{col}_Day'] = test[col].dt.day
    test[f'{col}_Hour'] = test[col].dt.hour
    test[f'{col}_Minute'] = test[col].dt.minute

# 원래 있던 시간 정보 열 삭제
test.drop(columns=time_columns, inplace=True)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE END POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE END POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'HEAD Standby Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['HEAD Standby Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'Head Clean Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['Head Clean Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE START POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE START POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE SPEED Collect', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE SPEED Collect'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# '-'를 기준으로 앞부분과 뒷부분을 분리
train['workorder'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[0])
train['workorder_num'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

test['workorder'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[0])
test['workorder_num'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

In [ ]:
train['1st Pressure'] = train['1st Pressure 1st Pressure Unit Time_AutoClave'] / train['1st Pressure Collect Result_AutoClave']
train['2nd Pressure'] = train['2nd Pressure Unit Time_AutoClave'] / train['2nd Pressure Collect Result_AutoClave']
train['3rd Pressure'] = train['3rd Pressure Unit Time_AutoClave'] / train['3rd Pressure Collect Result_AutoClave']
train['Chamber Temp AutoClave'] = train['Chamber Temp. Unit Time_AutoClave'] / train['Chamber Temp. Collect Result_AutoClave']

test['1st Pressure'] = test['1st Pressure 1st Pressure Unit Time_AutoClave'] / test['1st Pressure Collect Result_AutoClave']
test['2nd Pressure'] = test['2nd Pressure Unit Time_AutoClave'] / test['2nd Pressure Collect Result_AutoClave']
test['3rd Pressure'] = test['3rd Pressure Unit Time_AutoClave'] / test['3rd Pressure Collect Result_AutoClave']
test['Chamber Temp AutoClave'] = test['Chamber Temp. Unit Time_AutoClave'] / test['Chamber Temp. Collect Result_AutoClave']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Dispense Volume Collect Result_Fill1'] = train['Dispense Volume(Stage1) Collect Result_Fill1'] + train['Dispense Volume(Stage2) Collect Result_Fill1'] + train['Dispense Volume(Stage3) Collect Result_Fill1']

test['Total Dispense Volume Collect Result_Fill1'] = test['Dispense Volume(Stage1) Collect Result_Fill1'] + test['Dispense Volume(Stage2) Collect Result_Fill1'] + test['Dispense Volume(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Process Time'] = train['Machine Tact time Collect Result_Dam'] + train['Machine Tact time Collect Result_Fill1'] + train['Machine Tact time Collect Result_Fill2']

test['Total Process Time'] = test['Machine Tact time Collect Result_Dam'] + test['Machine Tact time Collect Result_Fill1'] + test['Machine Tact time Collect Result_Fill2']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Circle Result'] = train['Stage1 Circle2 Distance Speed Collect Result_Dam'] + train['Stage1 Circle1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Circle Result'] = test['Stage1 Circle2 Distance Speed Collect Result_Dam'] + test['Stage1 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage2 Circle Result'] = train['Stage2 Circle2 Distance Speed Collect Result_Dam'] + train['Stage2 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage2 Circle Result'] = test['Stage2 Circle2 Distance Speed Collect Result_Dam'] + test['Stage2 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage3 Circle Result'] = train['Stage3 Circle2 Distance Speed Collect Result_Dam'] + train['Stage3 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage3 Circle Result'] = test['Stage3 Circle2 Distance Speed Collect Result_Dam'] + test['Stage3 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
train['Total Production'] = train['Production Qty Collect Result_Dam'] + train['Production Qty Collect Result_Fill1'] + train['Production Qty Collect Result_Fill2']

test['Total Production'] = test['Production Qty Collect Result_Dam'] + test['Production Qty Collect Result_Fill1'] + test['Production Qty Collect Result_Fill2']

In [ ]:
train['Workorder_Dam_Middle'] = train['Workorder_Dam'].str.slice(3, 5)

test['Workorder_Dam_Middle'] = test['Workorder_Dam'].str.slice(3, 5)

In [ ]:
train['Workorder_Dam_First'] = train['Workorder_Dam'].str.slice(0, 2)

test['Workorder_Dam_First'] = test['Workorder_Dam'].str.slice(0, 2)

In [ ]:
# 변경할 열들의 리스트
columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 열의 문자 값을 0으로 변경
train[columns_to_replace] = train[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)
test[columns_to_replace] = test[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)

In [ ]:
train.drop(['Stage1 Line1 Distance Speed Collect Result_Dam', 'Stage1 Line2 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage1 Line1 Distance Speed Collect Result_Dam', 'Stage1 Line2 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train.drop(['Stage2 Line2 Distance Speed Collect Result_Dam', 'Stage2 Line3 Distance Speed Collect Result_Dam', 'Stage2 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage2 Line2 Distance Speed Collect Result_Dam', 'Stage2 Line3 Distance Speed Collect Result_Dam', 'Stage2 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train.drop(['Stage3 Line1 Distance Speed Collect Result_Dam', 'Stage3 Line2 Distance Speed Collect Result_Dam', 'Stage3 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage3 Line1 Distance Speed Collect Result_Dam', 'Stage3 Line2 Distance Speed Collect Result_Dam', 'Stage3 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train1 = train
test1 = test

### New 전처리

In [ ]:
# 기존 코드 유지
categorical_feat = train1.columns.tolist()
len(categorical_feat)

categorical_feat.remove('target')

train1[categorical_feat] = train1[categorical_feat].astype('str')
test1[categorical_feat] = test1[categorical_feat].astype('str')

# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols_1 = categorical_feat

In [ ]:
train1[categorical_feat] = train1[categorical_feat].astype('str')
test1[categorical_feat] = test1[categorical_feat].astype('str')

In [ ]:
# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols_1 = categorical_feat

In [ ]:
train1['target'] = train1['target'].replace({'Normal': 0, 'AbNormal': 1})

In [ ]:
stratify_columns = train1[["target", "Workorder_Dam_Middle"]]

In [ ]:
# 데이터를 stratified split
x_trn_1, x_val_1 = train_test_split(
    train1, test_size=0.2, random_state=42, stratify=stratify_columns)

In [ ]:
y_trn_1 = x_trn_1['target']
x_trn_1 = x_trn_1.drop(['target'], axis=1)

y_val_1 = x_val_1['target']
x_val_1 = x_val_1.drop(['target'], axis=1)

## 3. 모델 학습
* eval로 파라미터 찾기


In [ ]:
# predict = np.zeros((len(x_val), 2))

In [ ]:
# cat_model_1 = CatBoostClassifier(
#     loss_function='Logloss',
#     eval_metric='F1',
#     iterations=3147,
#     depth=5,
#     l2_leaf_reg=7.35,
#     min_data_in_leaf=7,
#     scale_pos_weight=7.97,
#     learning_rate= 0.01,
#     random_seed=42,
#     cat_features=cat_cols_1,
#     early_stopping_rounds=10000,
# )

In [ ]:
# cat_model_1.fit(x_trn_1, y_trn_1,
#     eval_set=(x_val_1, y_val_1), ## 3013
#     verbose=100)

In [ ]:
# pred = cat_model_1.predict_proba(x_val_1)[:, 1]
# predict[:, 0] = pred

In [ ]:
# result = cat_model_1.predict(x_val_1)

In [ ]:
# f1_val = f1_score(y_val_1, result, pos_label= 1)
# print('F1_val:', f1_val)

In [ ]:
# cat_model = CatBoostClassifier(
#     loss_function='Logloss',
#     eval_metric='F1',
#     iterations=1832,
#     depth=5,
#     l2_leaf_reg=7.35,
#     min_data_in_leaf=7,
#     scale_pos_weight=7.97,
#     learning_rate= 0.01,
#     random_seed=42,
#     cat_features=cat_cols,
#     early_stopping_rounds=1000,
# )

In [ ]:
# cat_model.fit(x_trn, y_trn,
#     eval_set=(x_val, y_val),
#     verbose=100)

In [ ]:
# pred = cat_model.predict_proba(x_val)[:, 1]
# predict[:, 1] = pred

In [ ]:
# weights = np.array([0.5, 0.5])

# # weighted average 계산
# weighted_avg = np.sum(predict * weights, axis=1)

# print(weighted_avg)

In [ ]:
# result = np.where(weighted_avg >= 0.5, 1, 0)
# sum(result)

### 모델 학습
* 찾은 파라미터로 전체 데이터로 학습

In [ ]:
cat_model_1 = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='F1',
    iterations=3147,
    depth=5,
    l2_leaf_reg=7.35,
    min_data_in_leaf=7,
    scale_pos_weight=7.97,
    learning_rate= 0.01,
    random_seed=42,
    cat_features=cat_cols_1,
    early_stopping_rounds=10000,
)

In [ ]:
cat_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='F1',
    iterations=1832,
    depth=5,
    l2_leaf_reg=7.35,
    min_data_in_leaf=7,
    scale_pos_weight=7.97,
    learning_rate= 0.01,
    random_seed=42,
    cat_features=cat_cols,
    early_stopping_rounds=1000,
)

In [ ]:
cat_model_1.fit(train1.drop(['target'], axis=1), train1['target'], verbose=500)

In [ ]:
cat_model.fit(train2.drop(['target'], axis=1), train2['target'], verbose=500)

In [ ]:
predict = np.zeros((len(test1), 2))

In [ ]:
pred = cat_model_1.predict_proba(test1)[:, 1]
predict[:, 0] = pred

In [ ]:
pred = cat_model.predict_proba(test2)[:, 1]
predict[:, 1] = pred

In [ ]:
weights = np.array([0.5, 0.5])

# weighted average 계산
weighted_avg = np.sum(predict * weights, axis=1)

print(weighted_avg)

In [ ]:
result = np.where(weighted_avg >= 0.5, 1, 0)
sum(result)

In [ ]:
result = cat_model_1.predict(test1)
result

In [ ]:
result = cat_model.predict(test2)
result

In [ ]:
from sklearn.model_selection import KFold

# K-fold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 각 폴드별 최적 threshold를 저장할 리스트
thresholds = []

# K-fold 학습 및 threshold 계산
for train_idx, val_idx in kf.split(train1):
    X_train, X_val = train1.drop(['target'], axis=1).iloc[train_idx], train1.drop(['target'], axis=1).iloc[val_idx]
    y_train, y_val = train1['target'].iloc[train_idx], train1['target'].iloc[val_idx]

    # 예측 (확률값)
    y_pred_prob = cat_model_1.predict_proba(X_val)[:, 1]

    # 여러 threshold에 대해 f1 score 계산
    best_threshold = 0.5
    best_f1 = 0
    for threshold in np.arange(0.1, 1.0, 0.01):
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_val, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    # 각 폴드의 최적 threshold 저장
    thresholds.append(best_threshold)

# 각 폴드의 최적 threshold의 평균 계산
thresholds

In [ ]:
mt = np.mean(thresholds)
mt

In [ ]:
pred = cat_model_1.predict_proba(test1)[:, 1]

In [ ]:
result = np.where(pred >= mt, 1, 0)
sum(result)

In [ ]:
result = np.where(pred >= 0.53, 1, 0)
sum(result)

In [ ]:
submission['target'] = result
submission

In [ ]:
submission['target'].replace({1: 'AbNormal', 0:'Normal'}, inplace=True)
submission['target'].value_counts()

In [ ]:
# 첫 번째 조건을 정의합니다
condition1 = (
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill1']) |
    (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Fill2']) |
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill2'])
)

# 두 번째 조건을 정의합니다
condition2 = (
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill1']) |
    (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Fill2']) |
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill2'])
)

# 두 조건을 합칩니다
combined_condition = condition1 | condition2

# test 데이터프레임에서 조건에 해당하는 행의 'target' 열 값을 'AbNormal'로 변경합니다
test.loc[combined_condition, 'target'] = 'AbNormal'

# 조건에 해당하는 행의 인덱스를 가져옵니다
abnormal_indices = test[combined_condition].index

# df_sub 데이터프레임에서 해당 인덱스의 'target' 열 값을 'AbNormal'로 변경합니다
# 'Set ID'를 인덱스로 사용한다고 가정합니다. 만약 다른 열을 사용한다면 이 부분을 수정해야 합니다.
submission.loc[submission.index.isin(abnormal_indices), 'target'] = 'AbNormal'

# 결과 확인
submission['target'].value_counts()

In [ ]:
submission.to_csv('submission.csv', index=False)